In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt

#modelling
from catboost import CatBoostClassifier

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import preprocessing_Origins, cleaning_processing, HenDailyVariable_Origins
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
#initialise parameters
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_matching = config.dico_matching
dico_night_hour = config.dico_night_hour
nbr_maxdur2beremoved = config.nbr_maxdur2beremoved
dico_garden_opening_hour = config.dico_garden_opening_hour
path_extracted_data_cleaning = os.path.join(path_extracted_data,'Cleaning')
path_extracted_data_cleaning_model = os.path.join(path_extracted_data_cleaning,'model')  
path_extracted_data_cleaning_rawvscleaned = os.path.join(path_extracted_data_cleaning,'Raw&Cleaned')  
#create a director if not existing
if not os.path.exists(path_extracted_data_cleaning_rawvscleaned):
    os.makedirs(path_extracted_data_cleaning_rawvscleaned)
path_extracted_data_cleaning_rawvscleaned_DF = os.path.join(path_extracted_data_cleaning_rawvscleaned,'var_expl')
if not os.path.exists(path_extracted_data_cleaning_rawvscleaned_DF):
    os.makedirs(path_extracted_data_cleaning_rawvscleaned_DF)

#feature for model
li_cont_select = config.li_cont_select
li_bin = config.li_bin
li_cat_select = config.li_cat_select

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
#name_method = 'ML-method'
name_method = 'TD-method'

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download data & model

In [5]:
##when the verification_daily_total_duration'==54000, i.e we restrict to the same amount of day light & data that model 
#was not trained on
date_min = dt.datetime(2020, 11, 13)
date_max = date_min+dt.timedelta(days=60)
date_max

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


datetime.datetime(2021, 1, 12, 0, 0)

#download data and add the necessary processing as the duration, next/previous zones etc
#steps also done in "1_PreprocessedTraining&TestingData"
START_TIME = time.perf_counter()
#min date is dt.datetime(2020, 10, 15) so that we dont have only birds not moving
df = cleaning_processing(date_min=date_min, date_max=date_max, config=config)
df['time'] = df['Timestamp'].map(lambda x: dt.datetime.time(x))
df.rename(columns={'Zone':'Trackingsystem_Zone'}, inplace=True)
df.to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,'df_records_initial_processed.csv'), sep=';', index=False)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
print(df.shape)
df.head(3)

In [6]:
model = CatBoostClassifier()
model.load_model(os.path.join(path_extracted_data_cleaning_model,'FINAL_Catboost'));

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data preprocessing

In [7]:
#for me reproducible examples, we will donload the file again here
df = pd.read_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,'df_records_initial_processed.csv'), sep=';',
                 parse_dates=['Timestamp','date'], dayfirst=True) 
df['is_dur_smaller_60sec'] = df['duration'].map(lambda x: x<60)
display(df['is_dur_smaller_60sec'].value_counts())
df['duration_bounded_mn'] = df['duration'].map(lambda x: min(x,60*60)/60)
df['next_duration_bounded_mn'] = df['next_duration'].map(lambda x: min(x,60*60)/60)
df['previous_duration_bounded_mn'] = df['previous_duration'].map(lambda x: min(x,60*60)/60)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


False    323673
True     283289
Name: is_dur_smaller_60sec, dtype: int64

(606962, 45)


,Timestamp,HenID,Trackingsystem_Zone,PenID,ts_order,TagID,signalstrength,system,time,date,...,zone2_match,zone3_match,zone4_match,zone3_match_exist,next2zone==Zone,previous2zone==Zone,is_dur_smaller_60sec,duration_bounded_mn,next_duration_bounded_mn,previous_duration_bounded_mn
0,2020-10-27 00:03:07,hen_100,4_Zone,pen5,105930,tag_121,6,3 - 5,00:03:07,2020-10-27,...,3_Zone,2_Zone,NoOtherZone,1,0,0,True,0.083333,17.666667,0.000000
1,2020-10-27 00:03:12,hen_100,5_Zone,pen5,105931,tag_121,9,3 - 5,00:03:12,2020-10-27,...,4_Zone,NoOtherZone,NoOtherZone,0,1,0,False,17.666667,0.116667,0.083333
2,2020-10-27 00:15:39,hen_73,4_Zone,pen4,105932,tag_41,4,3 - 5,00:15:39,2020-10-27,...,5_Zone,NoOtherZone,NoOtherZone,0,1,0,True,0.050000,4.950000,0.000000


### handle nan

In [8]:
#add info on nan values in the initial df, in order to replace at the end all prediction based on features involvig at least 
#one nan, by 1
#Indeed, for records with some nan features, we will predict them a 1 all the time its only 0,05% of the time , i.e. when its 
#first or last transition. Indeed, we dont want to loose any records!
df['nbr_nan'] = df.filter(li_cont_select+li_cat_select, axis=1).isnull().sum(axis=1).tolist()
display(df['nbr_nan'].value_counts())
#fillnan for simplicity, but we will remove those rows predictions anyway
#display BEFORE categoricalfeatures
display(df[li_cat_select].head(3))
df[li_cont_select] = df[li_cont_select].fillna(df[li_cont_select].mean().iloc[0])
#display AFTER categoricalfeatures
df[li_cat_select] = df[li_cat_select].fillna(df[li_cat_select].mode().iloc[0])
#show data
display(df[config.li_cont_select].head(3))
display(df[config.li_cat_select].head(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    606460
1       342
2       160
Name: nbr_nan, dtype: int64

,Trackingsystem_Zone,zone2_match,previous1_zone,next1_zone,system
0,4_Zone,3_Zone,NaN,5_Zone,3 - 5
1,5_Zone,4_Zone,4_Zone,3_Zone,3 - 5
2,4_Zone,5_Zone,NaN,3_Zone,3 - 5


,signalstrength,signalstzone2,duration_bounded_mn,next_duration_bounded_mn,previous_duration_bounded_mn,next2zone==Zone,previous2zone==Zone,zone3_match_exist
0,6,5,0.083333,17.666667,0.000000,0,0,1
1,9,6,17.666667,0.116667,0.083333,1,0,0
2,4,3,0.050000,4.950000,0.000000,1,0,0


,Trackingsystem_Zone,zone2_match,previous1_zone,next1_zone,system
0,4_Zone,3_Zone,3_Zone,5_Zone,3 - 5
1,5_Zone,4_Zone,4_Zone,3_Zone,3 - 5
2,4_Zone,5_Zone,3_Zone,3_Zone,3 - 5


# Cleaning

In [9]:
if name_method=='ML-method':
    START_TIME = time.perf_counter()
    #predict, and if its wintergarten (i.e. 1_Zone) put =1
    df['prediction'] = df.apply(lambda x: model.predict(pd.DataFrame(x[li_cont_select+\
                                                                             li_cat_select]).transpose())[0] if\
                                      x['Trackingsystem_Zone']!='1_Zone' else 1, axis=1)
    #predict false if <60seconds in WG
    df.loc[(df['is_dur_smaller_60sec'])&(df['Trackingsystem_Zone']=='1_Zone'), 'prediction'] = 0
    #always =1 if there was some nan values
    df.loc[df['nbr_nan']!=0, 'prediction'] = 1
    END_TIME = time.perf_counter()
    print ("Total running time for prediction: %.2f mn" %((END_TIME-START_TIME)/60))

    #show info & save (take TIME to save)
    display(display(df.head(3)))
    display(df['prediction'].value_counts())
    display(df['prediction'].value_counts(normalize=True)) #in testing it was 4,9% of wrong records: 42/(42+821)*100)
    df.filter(['HenID','PenID','system','Trackingsystem_Zone','prediction','Timestamp','date'],
              axis=1).to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned, id_run+name_method+'_Rawdata&Model.csv'),
                             index=False, sep=';')    
    
elif name_method=='TD-method':
    #same but with TD
    #predict, and if its wintergarten (i.e. 1_Zone) put =1
    df['prediction'] = df['duration'].map(lambda x: 0 if x<60 else 1)
    #always =1 if there was some nan values
    df.loc[df['nbr_nan']!=0, 'prediction'] = 1
    #show info & save (take TIME to save)
    display(display(df.head(3)))
    display(df['prediction'].value_counts())
    display(df['prediction'].value_counts(normalize=True)) #in testing it was 4,9% of wrong records: 42/(42+821)*100)
    df.filter(['HenID','PenID','system','Trackingsystem_Zone','prediction','Timestamp','date'],
              axis=1).to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned, id_run+name_method+'_Rawdata&Model.csv'),
                             index=False, sep=';')  
    
else:
    print('choose an existing method (ML-method, or TD-method)')
    sys.exit()

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Timestamp,HenID,Trackingsystem_Zone,PenID,ts_order,TagID,signalstrength,system,time,date,...,zone4_match,zone3_match_exist,next2zone==Zone,previous2zone==Zone,is_dur_smaller_60sec,duration_bounded_mn,next_duration_bounded_mn,previous_duration_bounded_mn,nbr_nan,prediction
0,2020-10-27 00:03:07,hen_100,4_Zone,pen5,105930,tag_121,6,3 - 5,00:03:07,2020-10-27,...,NoOtherZone,1,0,0,True,0.083333,17.666667,0.000000,1,1
1,2020-10-27 00:03:12,hen_100,5_Zone,pen5,105931,tag_121,9,3 - 5,00:03:12,2020-10-27,...,NoOtherZone,0,1,0,False,17.666667,0.116667,0.083333,0,1
2,2020-10-27 00:15:39,hen_73,4_Zone,pen4,105932,tag_41,4,3 - 5,00:15:39,2020-10-27,...,NoOtherZone,0,1,0,True,0.050000,4.950000,0.000000,1,1


None

1    323845
0    283117
Name: prediction, dtype: int64

1    0.533551
0    0.466449
Name: prediction, dtype: float64

# Compute variable

#IF NOT ALREADY DONE
p = glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\TagUpdates\log*'))
#print('There is %d log files:\n  %s'%(len(p),'  \n  '.join(p)))
df = preprocessing_Origins(p, config, save=True, dodevice=True)
print(df.shape)
display(df.head(2))
display(df.tail(2))

In [10]:
#for being more reproductible, we open the file that was saved from cleaning
df = pd.read_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned, id_run+name_method+'_Rawdata&Model.csv'), sep=';', 
                 parse_dates=['Timestamp','date'], dayfirst=True)
df.rename(columns={'Trackingsystem_Zone':'Zone'}, inplace=True)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(606962, 7)


,HenID,PenID,system,Zone,prediction,Timestamp,date
0,hen_100,pen5,3 - 5,4_Zone,1,2020-10-27 00:03:07,2020-10-27
1,hen_100,pen5,3 - 5,5_Zone,1,2020-10-27 00:03:12,2020-10-27
2,hen_73,pen4,3 - 5,4_Zone,1,2020-10-27 00:15:39,2020-10-27


In [11]:
############## cleaned dataset
START_TIME = time.perf_counter()
#remove predicted-wrong-records
print(df.shape)
df_ = df[df['prediction']!=0].copy()
print(df_.shape)
#simply rename the tracking system zone into the model zone 
display(df_.head(2))
for p in tqdm.tqdm(df_['PenID'].unique()):
    df_daily = HenDailyVariable_Origins(df_[df_['PenID']==p], config, name_='CLEANING_'+name_method+p, 
                                        timestamp_name='Timestamp', has_cons_equal_zone=True)
    print(df_daily.shape)
    df_daily.head(3)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

#Concatenate all HenVariables csv into one
li_paths_var = glob.glob(os.path.join(path_extracted_data, id_run+'_daily__CLEANING_'+name_method+'*_variables.csv'))
li_df = []
for path_var in tqdm.tqdm(li_paths_var):
    #for being more reproductible, we open the file that was saved from cleaning
    df_ = pd.read_csv(path_var, sep=';',parse_dates=['level', 'FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 
                                                     'FirstTimestamp_3_Zone','FirstTimestamp_4_Zone',
                                                     'FirstTimestamp_5_Zone']) 
    print(df_.shape)
    li_df.append(df_)
df_daily = pd.concat(li_df)
df_daily.to_csv(os.path.join(path_extracted_data, id_run+name_method+'_daily_CLEANING_ALL_variables.csv'), sep=';', index=False)
print(df_daily.shape)
df_daily.head(3)

(606962, 7)
(323845, 7)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,HenID,PenID,system,Zone,prediction,Timestamp,date
0,hen_100,pen5,3 - 5,4_Zone,1,2020-10-27 00:03:07,2020-10-27
1,hen_100,pen5,3 - 5,5_Zone,1,2020-10-27 00:03:12,2020-10-27


  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values([timestamp_name], inplace=True)
C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:3942: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[timestamp_name] = df[timestamp_name].map(lambda x: dt.datetime(x.year,x.month,x.day,x.hour,x.minute, x.second))


----------------- Create time serie
in this time series there is 20 hens
The initial starting date in over all is: 2020-10-27 00:03:07, and the ending date will be: 2020-11-22 23:10:43
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-10-27 00:03:07, and the ending date will be: 2020-11-22 23:59:59
Total running time: 0.55 mn
your time series has 1 seconds between two timestamps
----------------- main night zone and nbr of transitions over night....
----------------- total duration per Zone in seconds....
----------------- first timestamp in each zone per day....
----------------- number of Zone (excluding nan)....
----------------- compute some variables based on a list of zones over a day....
----------------NESTBOX


...

(514, 124)


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.04it/s]

(566, 124)


(4257, 124)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,list_of_night20_2_temperature,nbr_temperature_nnnan,temperature_night20_2_median,temperature_night20_2_max,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean
0,hen_131,2020-10-27,2393.0,20613.0,16026.0,3296.0,872.0,43200.0,"[2393.0, 20613.0, 16026.0, 3296.0, 872.0]","[0.055393518518518516, 0.47715277777777776, 0....",...,"[29.0, 29.0, 30.0, 29.0, 29.0, 29.0, 29.0, 30....",36.0,29.0,30.0,0.212191,"[41.0, 62.0, 72.0, 71.0, 63.0, 50.0, 36.0, 39....",72.0,0.0,103.0,67.236111
1,hen_131,2020-10-28,3482.0,19982.0,16698.0,2635.0,403.0,43200.0,"[3482.0, 19982.0, 16698.0, 2635.0, 403.0]","[0.08060185185185186, 0.4625462962962963, 0.38...",...,"[28.0, 28.0, 28.0, 28.0, 28.0, 29.0, 29.0, 28....",36.0,29.0,29.0,0.212191,"[45.0, 78.0, 59.0, 64.0, 52.0, 43.0, 43.0, 55....",72.0,0.0,109.0,69.138889
2,hen_131,2020-10-29,2768.0,21508.0,15045.0,2791.0,1088.0,43200.0,"[2768.0, 21508.0, 15045.0, 2791.0, 1088.0]","[0.06407407407407407, 0.49787037037037035, 0.3...",...,"[29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29....",36.0,29.0,29.0,0.076389,"[66.0, 61.0, 59.0, 68.0, 59.0, 73.0, 58.0, 42....",72.0,0.0,108.0,71.847222


In [12]:
############## raw dataset
#open time series per session and compute the variables for each session (car time series make sence at session level), et en 
#plus des variables tel que running entropy over the whole session ateach last timestamp of each level make sence only at 
#session level
#once to much data, do this per pen!!! with name=pens!!
START_TIME = time.perf_counter()
for p in tqdm.tqdm(df['PenID'].unique()):
    df_daily = HenDailyVariable_Origins(df[df['PenID']==p], config, name_='RAW_'+name_method+p, timestamp_name='Timestamp',
                                       has_cons_equal_zone=True)
    print(df_daily.shape)
    df_daily.head(3)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

#Concatenate all HenVariables csv into one
li_paths_var = glob.glob(os.path.join(path_extracted_data, id_run+'_daily__RAW_'+name_method+'*_variables.csv'))
li_df = []
for path_var in tqdm.tqdm(li_paths_var):
    #for being more reproductible, we open the file that was saved from cleaning
    df_ = pd.read_csv(path_var, sep=';',parse_dates=['level', 'FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 
                                                     'FirstTimestamp_3_Zone','FirstTimestamp_4_Zone',
                                                     'FirstTimestamp_5_Zone']) 
    print(df_.shape)
    li_df.append(df_)
df_daily = pd.concat(li_df)
df_daily.to_csv(os.path.join(path_extracted_data, id_run+name_method+'_daily_RAW_ALL_variables.csv'), sep=';', index=False)
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values([timestamp_name], inplace=True)
C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:3942: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

----------------- Create time serie
in this time series there is 20 hens
The initial starting date in over all is: 2020-10-27 00:03:07, and the ending date will be: 2020-11-22 23:10:43
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-10-27 00:03:07, and the ending date will be: 2020-11-22 23:59:59
Total running time: 0.45 mn
your time series has 1 seconds between two timestamps
----------------- main night zone and nbr of transitions over night....
----------------- total duration per Zone in seconds....
----------------- first timestamp in each zone per day....
----------------- number of Zone (excluding nan)....
----------------- compute some variables based on a list of zones over a day....
----------------NESTBOX



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  8.25it/s]
C:\Users\camil\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1366: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


------------ add hen basics info
From the focalBirdinfo, you have 160 ative tags
(540, 110)
(527, 112)
-------------- Lets remove unwanted dates at PENS level


All the night variables are:  Index(['HenID', 'night_level', 'night_Max_duration_zones',
       'night_distribution_entropy', 'night_Total_number_transition', 'level',
       'PenID', 'CLASS', 'R-Pen', 'InitialStartDate', '29-09 weight',
       '10-12 juin weight', 'weight 23-11-2020', 'weight 04-01-2021',
       'weight 01-02-21', 'weight 12-04-21', 'Treatment'],
      dtype='object')
['night_level']
-------------- Lets remove dates of tags when they were not giving deviceupdate regularly


...

(514, 124)


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:18<00:00,  2.29s/it]

(566, 124)


(4257, 124)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,list_of_night20_2_temperature,nbr_temperature_nnnan,temperature_night20_2_median,temperature_night20_2_max,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean
0,hen_131,2020-10-27,2258.0,19862.0,17004.0,3204.0,872.0,43200.0,"[2258.0, 19862.0, 17004.0, 3204.0, 872.0]","[0.05226851851851852, 0.4597685185185185, 0.39...",...,"[29.0, 29.0, 30.0, 29.0, 29.0, 29.0, 29.0, 30....",36.0,29.0,30.0,0.212191,"[41.0, 62.0, 72.0, 71.0, 63.0, 50.0, 36.0, 39....",72.0,0.0,103.0,67.236111
1,hen_131,2020-10-28,3669.0,19439.0,17185.0,2504.0,403.0,43200.0,"[3669.0, 19439.0, 17185.0, 2504.0, 403.0]","[0.08493055555555555, 0.4499768518518519, 0.39...",...,"[28.0, 28.0, 28.0, 28.0, 28.0, 29.0, 29.0, 28....",36.0,29.0,29.0,0.212191,"[45.0, 78.0, 59.0, 64.0, 52.0, 43.0, 43.0, 55....",72.0,0.0,109.0,69.138889
2,hen_131,2020-10-29,2741.0,21189.0,15548.0,2727.0,995.0,43200.0,"[2741.0, 21189.0, 15548.0, 2727.0, 995.0]","[0.06344907407407407, 0.4904861111111111, 0.35...",...,"[29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29....",36.0,29.0,29.0,0.076389,"[66.0, 61.0, 59.0, 68.0, 59.0, 73.0, 58.0, 42....",72.0,0.0,108.0,71.847222


In [13]:
#saving both files into one
#for being more reproductible, we open the file that was saved from cleaning
df_c = pd.read_csv(os.path.join(path_extracted_data, id_run+name_method+'_daily_CLEANING_ALL_variables.csv'), sep=';', 
                   parse_dates=['level'], dayfirst=True) 
df_c['type'] = 'cleaning'
df_r = pd.read_csv(os.path.join(path_extracted_data, id_run+name_method+'_daily_RAW_ALL_variables.csv'), sep=';', 
                   parse_dates=['level'], dayfirst=True) 
df_r['type'] = 'raw'
df_var = pd.concat([df_c,df_r])
df_var.to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,name_method+'_daily_ALL_variables.csv'), sep=';', 
              index=False)
print(df_r.shape,df_c.shape,df_var.shape)
print(df_var.shape)
df_var.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(4257, 125) (4257, 125) (8514, 125)
(8514, 125)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,nbr_temperature_nnnan,temperature_night20_2_median,temperature_night20_2_max,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean,type
0,hen_131,2020-10-27,2393.0,20613.0,16026.0,3296.0,872.0,43200.0,"[2393.0, 20613.0, 16026.0, 3296.0, 872.0]","[0.055393518518518516, 0.47715277777777776, 0....",...,36.0,29.0,30.0,0.212191,"[41.0, 62.0, 72.0, 71.0, 63.0, 50.0, 36.0, 39....",72.0,0.0,103.0,67.236111,cleaning
1,hen_131,2020-10-28,3482.0,19982.0,16698.0,2635.0,403.0,43200.0,"[3482.0, 19982.0, 16698.0, 2635.0, 403.0]","[0.08060185185185186, 0.4625462962962963, 0.38...",...,36.0,29.0,29.0,0.212191,"[45.0, 78.0, 59.0, 64.0, 52.0, 43.0, 43.0, 55....",72.0,0.0,109.0,69.138889,cleaning
2,hen_131,2020-10-29,2768.0,21508.0,15045.0,2791.0,1088.0,43200.0,"[2768.0, 21508.0, 15045.0, 2791.0, 1088.0]","[0.06407407407407407, 0.49787037037037035, 0.3...",...,36.0,29.0,29.0,0.076389,"[66.0, 61.0, 59.0, 68.0, 59.0, 73.0, 58.0, 42....",72.0,0.0,108.0,71.847222,cleaning


In [14]:
min(df_var['level'].unique()), max(df_var['level'].unique())

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(numpy.datetime64('2020-10-26T00:00:00.000000000'),
 numpy.datetime64('2020-11-22T00:00:00.000000000'))

In [15]:
print(list(df_var.columns))

['HenID', 'level', 'duration_1_Zone', 'duration_2_Zone', 'duration_3_Zone', 'duration_4_Zone', 'duration_5_Zone', 'verification_daily_total_duration', 'dur_values', 'dur_values_normalized', 'FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 'FirstTimestamp_3_Zone', 'FirstTimestamp_4_Zone', 'FirstTimestamp_5_Zone', 'Total_number_zone', 'list_of_durations', 'list_of_zones', 'Max_duration_zones', 'dico_zone_sortedduration', 'Total_number_transition', 'nbr_stays', 'distribution_entropy', 'vertical_travel_distance', 't_DU_missingZone_mvtPerc', 'li_event_chaoticmvt_z_d', 'nbr_stays_3_Zone', 'nbr_stays_2_Zone', 'nbr_stays_4_Zone', 'nbr_stays_5_Zone', 'nbr_stays_1_Zone', 'Max_duration_zone_4', 'down_missingZone_mvtPerc', 'up_missingZone_mvtPerc', 'mean_duration', 'median_duration', 'max_duration', 'min_duration', 'variance_duration', 'percentile_5', 'percentile_15_duration', 'percentile_85_duration', 'percentile_95_duration', 'median_abs_deviation_duration', 'kurtosis_duration', 'skew_duration'

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
